In [32]:
from torchvision import datasets, transforms
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Functions

In [33]:
logistic = lambda z: 1./ (1 + np.exp(-z))
relu = lambda z: np.maximum(0, z)

In [60]:
train_dataset = datasets.FashionMNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transforms.ToTensor()
    )
test_dataset = datasets.FashionMNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transforms.ToTensor()
    )

# get loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
    )
test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=64, 
    shuffle=True
)

In [ ]:
# TODO: find better way to normalize dataset

def compute_loader_stats(loader, return_distribution=False):
    n_pixels = 0
    sum_pixels = 0
    sum_squared = 0
    
    dist = []
    for images, _ in loader:
        n_pixels += images.numel()
        sum_pixels += images.sum().item()
        sum_squared += (images ** 2).sum().item()
        if return_distribution:
            dist.append(images.numpy().flatten())
        
    mean = sum_pixels / n_pixels
    var = (sum_squared / n_pixels) - (mean ** 2)
    std = np.sqrt(var)
    return mean, std, dist if return_distribution else (mean, std)


def compute_data_stats(*loaders, return_distribution=False):
    total_mean = 0
    total_std = 0
    for loader in loaders:
        stats = compute_loader_stats(loader, return_distribution=return_distribution)
        total_mean += stats[0]
        total_std += stats[1]
        print(f'Loader mean: {stats[0]:.4f}, std: {stats[1]:.4f}')
    n_loaders = len(loaders)
    print(f'Overall mean: {total_mean / n_loaders:.4f}, std: {total_std / n_loaders:.4f}')
    total_mean /= n_loaders
    total_std /= n_loaders
    return total_mean, total_std, stats[2] if return_distribution else (total_mean, total_std)

In [61]:
# temp variables until normalization done
norm_train = train_dataset
norm_test = test_dataset

In [ ]:
# stats = compute_data_stats(train_loader, test_loader, return_distribution=True)
# mean, std = stats[0], stats[1]
# print(f'Final mean: {mean:.4f}, std: {std:.4f}')

# distribution_prenorm = np.concatenate(stats[2])

In [56]:
# sns.histplot(distribution_prenorm, bins=100, kde=True)
# plt.yscale('log')
# plt.title('Pixel Value Distribution Before Normalization')

In [55]:
# # normalize datasets
# train_dataset.transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((mean,), (std,))
# ])

# test_dataset.transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((mean,), (std,))
# ])

In [62]:
# stats = compute_data_stats(train_loader, test_loader, return_distribution=True)
# mean_postnorm, std_postnorm = stats[0], stats[1]
# print(f'Final mean: {mean_postnorm:.4f}, std: {std_postnorm:.4f}')

# distribution_postnorm = np.concatenate(stats[2])

In [58]:
# sns.histplot(distribution_postnorm, bins=100, kde=True)
# plt.yscale('log')
# plt.title('Pixel Value Distribution Before Normalization')

In [65]:
class MLP:
    """
    Constructor takes:
    g = output activation function
    h = hidden activation function
    L = number of hidden layers
    M = number of hidden units, iterable (each element corresponds to a layer)
    
    -> Weights and biases are initialized randomly
    """
    def __init__(self, g=relu, h=relu, L=1, M = 64): 
        self.g = g
        self.h = h
        self.L = L
        
        if isinstance(M, int):
            self.M = M
        else:
            assert len(M) == L, "Length of M must equal L"
            self.M = M
        
    def fit(self, x, y, optimizer):
        N,D = x.shape
        def gradient(x, y, params):
            v, w = params
            z = logistic(np.dot(x, v)) #N x M
            yh = logistic(np.dot(z, w))#N
            dy = yh - y #N
            dw = np.dot(z.T, dy)/N #M
            dz = np.outer(dy, w) #N x M
            dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
            dparams = [dv, dw]
            return dparams
        
        w = np.random.randn(self.M) * .01
        v = np.random.randn(D,self.M) * .01
        params0 = [v,w]
        self.params = optimizer.run(gradient, x, y, params0)
        return self
    
    def predict(self, x):
        v, w = self.params
        z = logistic(np.dot(x, v)) #N x M
        yh = logistic(np.dot(z, w))#N
        return yh

In [64]:
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
            grad = gradient_fn(x, y, params)
            for p in range(len(params)):
                params[p] -= self.learning_rate * grad[p]
            t += 1
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params